# 公式のkernel

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


import pandas as pd
import numpy as np
from collections import Counter

n_children = 1000000 # n children to give
n_gift_type = 1000 # n types of gifts available
n_gift_quantity = 1000 # each type of gifts are limited to this quantity
n_gift_pref = 10 # number of gifts a child ranks
n_child_pref = 1000 # number of children a gift ranks
twins = int(0.004 * n_children)    # 0.4% of all population, rounded to the closest even number
ratio_gift_happiness = 2
ratio_child_happiness = 2


gift_pref = pd.read_csv('../input/child_wishlist.csv',header=None).drop(0, 1).values
child_pref = pd.read_csv('../input/gift_goodkids.csv',header=None).drop(0, 1).values


def avg_normalized_happiness(pred, child_pref, gift_pref):
    
    # check if number of each gift exceeds n_gift_quantity
    gift_counts = Counter(elem[1] for elem in pred)
    for count in gift_counts.values():
        assert count <= n_gift_quantity
                
    # check if twins have the same gift
    for t1 in range(0,twins,2):
        twin1 = pred[t1]
        twin2 = pred[t1+1]
        assert twin1[1] == twin2[1]
    
    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for row in pred:
        child_id = row[0]
        gift_id = row[1]
        
        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0 
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(gift_pref[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = ( n_child_pref - np.where(child_pref[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
    
    # print(max_child_happiness, max_gift_happiness
    print('normalized child happiness=',float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) , \
        ', normalized gift happiness',np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity))
    return float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) + np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity)

random_sub = pd.read_csv('../input/sample_submission_random.csv').values.tolist()
print(avg_normalized_happiness(random_sub, child_pref, gift_pref))

child_wishlist.csv
gift_goodkids.csv
sample_submission_random.csv

normalized child happiness= -0.04404195 , normalized gift happiness 1.18475e-05
-0.0440301025


# スタート(kernel) 子どもが貪欲にとる

In [2]:
# coding: utf-8
__author__ = 'ZFTurbo: https://kaggle.com/zfturbo'

import pandas as pd
import numpy as np

INPUT_PATH = '../input/'


def avg_normalized_happiness(pred, child_pref, gift_pref):
    n_children = 1000000  # n children to give
    n_gift_type = 1000  # n types of gifts available
    n_gift_quantity = 1000  # each type of gifts are limited to this quantity
    n_gift_pref = 10  # number of gifts a child ranks
    n_child_pref = 1000  # number of children a gift ranks
    twins = int(0.004 * n_children)  # 0.4% of all population, rounded to the closest even number
    ratio_gift_happiness = 2
    ratio_child_happiness = 2

    # check if twins have the same gift
    for t1 in range(0, twins, 2):
        twin1 = pred[t1]
        twin2 = pred[t1 + 1]
        assert twin1 == twin2

    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)

    for i in range(len(pred)):
        child_id = i
        gift_id = pred[i]

        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(gift_pref[child_id] == gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = (n_child_pref - np.where(child_pref[gift_id] == child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness

    # print(max_child_happiness, max_gift_happiness
    print('Normalized child happiness: ', float(total_child_happiness) / (float(n_children) * float(max_child_happiness)))
    print('Normalized santa happiness: ', np.mean(total_gift_happiness) / float(max_gift_happiness * n_gift_quantity))
    return float(total_child_happiness) / (float(n_children) * float(max_child_happiness)) \
           + np.mean(total_gift_happiness) / float(max_gift_happiness * n_gift_quantity)
           
           
def solve():
    wish = pd.read_csv(INPUT_PATH + 'child_wishlist.csv', header=None).as_matrix()[:, 1:]
    gift = pd.read_csv(INPUT_PATH + 'gift_goodkids.csv', header=None).as_matrix()[:, 1:]
    answ = np.zeros((len(wish)), dtype=np.int32)
    answ[:] = -1
    gift_count = np.zeros((len(gift)), dtype=np.int32)

    for i in range(0, 4000, 2):
        g = wish[i, 0]
        answ[i] = g
        answ[i+1] = g
        gift_count[g] += 2

    for i in range(4000, len(answ)):
        for k in range(10):
            g = wish[i, k]
            if gift_count[g] < 1000:
                answ[i] = g
                gift_count[g] += 1
                break
        if answ[i] == -1:
            g = np.argmin(gift_count)
            answ[i] = g
            gift_count[g] += 1

        if i % 100000 == 0:
            print('Completed: {}'.format(i))

    if gift_count.max() > 1000:
        print('Some error in kernel: {}'.format(gift_count.max()))

    score = avg_normalized_happiness(answ, gift, wish)
    print('Predicted score: {:.8f}'.format(score))

    out = open('../submit/subm.csv', 'w')
    out.write('ChildId,GiftId\n')
    for i in range(len(answ)):
        out.write(str(i) + ',' + str(answ[i]) + '\n')
    out.close()


if __name__ == '__main__':
    solve()

Completed: 100000
Completed: 200000
Completed: 300000
Completed: 400000
Completed: 500000
Completed: 600000
Completed: 700000
Completed: 800000
Completed: 900000
Normalized child happiness:  0.81689025
Normalized santa happiness:  -5.7185e-06
Predicted score: 0.81688453


In [5]:
gift_pref.shape

(1000000, 10)

In [4]:
child_pref.shape

(1000, 1000)

### Child class

Gift のpreferenceもchildに持たせる

In [47]:
class Child(object):
    
    def __init__(self, idx, prefer):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for i in range(prefer.shape[0]):
            self.prefer_dict[prefer[i]] = 200*(prefer.shape[0] - i) - 1
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid] += score + 1
        else:
            self.prefer_dict[giftid] = score - 100
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, -101)       

In [48]:
Children = [Child(i, gift_pref[i]) for i in range(1000000)]

In [49]:
for j in range(1000):
    cf = child_pref[j]
    for i in range(cf.shape[0]):
        Children[cf[i]].add_gifts_prefer(j, 2*(cf.shape[0] - i))

### Gifts
今のところただのlistだがそのうち色々作ってメモさせる  
item kが来たら、1人なら誰を、2人なら誰を（双子はこの時だけ選択）shuffleに出したらどれだけ特か、とか

In [50]:
Gifts_list = [[] for i in range(1000)]

初期値

In [51]:
pred_greedy = pd.read_csv('../submit/subm.csv').values.tolist()

In [52]:
for p in pred_greedy:
    Gifts_list[p[1]].append(p[0])

In [57]:
def diff_child(c, i, j):
    # c:child, i,j : item
    # returns : diff happiness if gift for child c is changed from i to j
    if c >= 4000:
        return Children[c].happiness(i) - Children[c].happiness(j)
    # twins
    elif c % 2 == 0:
        return (Children[c].happiness(i) + Children[c+1].happiness(i) - Children[c].happiness(j) - Children[c+1].happiness(j)) / 2
    else:
        return (Children[c].happiness(i) + Children[c-1].happiness(i) - Children[c].happiness(j) - Children[c-1].happiness(j)) / 2

In [82]:
def swap_greedy(i, j):
    # i, j : item
    # get current children who are going to be presented gift i or j
    # returns : swaped list
    # there are some bugs in handling twins!
    
    all_users = Gifts_list[i] + Gifts_list[j]
    prefer_order = np.argsort([diff_child(c, i, j) for c in all_users])
    j_list = [all_users[k] for k in prefer_order[:1000]]
    i_list = [all_users[k] for k in prefer_order[1000:]]
    
    # care when twins split
    if j_list[-1] < 4000 and i_list[0] // 2 == j_list[-1] // 2:
        # twins split
        # redo with sort
        all_users = Gifts_list[i] + Gifts_list[j]
        all_users.sort()
        prefer_order = np.argsort([diff_child(c, i, j) for c in all_users])
        j_list = [all_users[k] for k in prefer_order[:1000]]
        i_list = [all_users[k] for k in prefer_order[1000:]]
        
        if j_list[-1] < 4000 and i_list[0] // 2 == j_list[-1] // 2:
            # still twins split
            diff_c_twins = diff_child(j_list[-1], i, j)
            # loss when the twins moves j
            loss_j = 1000000
            for q in range(999, -1, -1):
                if diff_child(j_list[q], i, j) <= diff_c_twins and j_list[q] < 4000:
                    loss_j = diff_c_twins - diff_child(j_list[q], i, j)
                    break
            # loss when the twins moves i
            loss_i = 1000000
            for p in range(1000):
                if diff_child(i_list[p], i, j) >= diff_c_twins and i_list[p] < 4000:
                    loss_i = diff_child(i_list[p], i, j) - diff_c_twins
                    break

            # run smaller
            if loss_i <= loss_j:
                i_list.append(j_list.pop())
                j_list.append(i_list.pop(p))
            else:
                j_list.append(i_list.pop(0))
                i_list.append(j_list.pop(q))
                
    return i_list, j_list
            
        

# Main

In [102]:
np.random.seed(57)
add_res = 0
for t in range(1000000):
    i, j = np.random.choice(1000, 2, replace=False)
    i_list, j_list = swap_greedy(i, j)
    now_happiness = sum([Children[c].happiness(i) for c in Gifts_list[i]]) + sum([Children[c].happiness(j) for c in Gifts_list[j]])
    new_happiness = sum([Children[c].happiness(i) for c in i_list]) + sum([Children[c].happiness(j) for c in j_list])
    
    if now_happiness < new_happiness:
        add_res += new_happiness - now_happiness
        Gifts_list[i] = i_list
        Gifts_list[j] = j_list
        
    if t % 1000 == 0:
        print("t = " + str(t) +" : increased " + str(add_res))





t = 0 : increased 1511
t = 1000 : increased 1421065
t = 2000 : increased 2726219
t = 3000 : increased 4030903
t = 4000 : increased 5209464
t = 5000 : increased 6423767
t = 6000 : increased 7575639
t = 7000 : increased 8656540
t = 8000 : increased 9761242
t = 9000 : increased 10922110
t = 10000 : increased 12003484
t = 11000 : increased 13046515
t = 12000 : increased 14112360
t = 13000 : increased 15150819
t = 14000 : increased 16112651
t = 15000 : increased 17074230
t = 16000 : increased 17999286
t = 17000 : increased 19063394
t = 18000 : increased 19970911
t = 19000 : increased 20824745
t = 20000 : increased 21725956
t = 21000 : increased 22672945
t = 22000 : increased 23546231
t = 23000 : increased 24437648
t = 24000 : increased 25260074
t = 25000 : increased 26166728
t = 26000 : increased 27062080
t = 27000 : increased 27903417
t = 28000 : increased 28737619
t = 29000 : increased 29597696
t = 30000 : increased 30442876
t = 31000 : increased 31229847
t = 32000 : increased 32031305
t 

In [103]:
# greedy kernel
1633769063

1633769063

In [104]:
1633769063 + add_res

1838780074

In [105]:
res_list_temp = [[] for i in range(1000000)]
for j in range(1000):
    for i in Gifts_list[j]:
        res_list_temp[i] = [i, j]

バグのためassert twin1[1] == twin2[1] がこけるので、評価関数を変える

In [154]:
def avg_normalized_happiness_(pred, child_pref, gift_pref):
    
    # check if number of each gift exceeds n_gift_quantity
    gift_counts = Counter(elem[1] for elem in pred)
    for count in gift_counts.values():
        assert count <= n_gift_quantity
                
    # check if twins have the same gift
    # for t1 in range(0,twins,2):
    #     twin1 = pred[t1]
    #     twin2 = pred[t1+1]
    #     assert twin1[1] == twin2[1]
    
    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for row in pred:
        child_id = row[0]
        gift_id = row[1]
        
        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0 
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(gift_pref[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = ( n_child_pref - np.where(child_pref[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
    
    # print(max_child_happiness, max_gift_happiness
    print('normalized child happiness=',float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) , \
        ', normalized gift happiness',np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity))
    return float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) + np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity)



In [108]:
avg_normalized_happiness_(res_list_temp, child_pref, gift_pref)

normalized child happiness= 0.8566427 , normalized gift happiness 0.062747337


0.91939003699999999

In [116]:
# modify for twins
twins_gift_cnt = [0 for i in range(1000)]
for i in range(4000):
    twins_gift_cnt[res_list_temp[i][1]] += 1
    
res_list_temp_2 = [[] for i in range(1000000)]



In [117]:
# Greedy from first
for i in range(2000):
    res_hap = -1000000
    for j in range(1000):
        if twins_gift_cnt[j] > 0:
            res_hap_ = Children[2*i].happiness(j) + Children[2*i+1].happiness(j)
            if res_hap_ > res_hap:
                jmax = j
    res_list_temp_2[2*i] = [2*i, jmax]
    res_list_temp_2[2*i+1] = [2*i+1, jmax]
    twins_gift_cnt[jmax] -= 2
    
for i in range(4000, 1000000):
    if twins_gift_cnt[res_list_temp[i][1]] >= 0:
        res_list_temp_2[i] = res_list_temp[i]
    else:
        for j in range(1000):
            if twins_gift_cnt[j] > 0:
                res_hap_ = Children[i].happiness(j) + Children[i].happiness(j)
                if res_hap_ > res_hap:
                    jmax = j
        res_list_temp_2[i] = [i, jmax]
        twins_gift_cnt[jmax] -= 1
        twins_gift_cnt[res_list_temp[i][1]] += 1
        

In [120]:
avg_normalized_happiness(res_list_temp_2, child_pref, gift_pref)

normalized child happiness= 0.85591425 , normalized gift happiness 0.0625118425


0.91842609249999996

In [121]:
out = open('../submit/subm_hrd1213_1000000.csv', 'w')
out.write('ChildId,GiftId\n')
for i in range(len(res_list_temp_2)):
    out.write(str(i) + ',' + str(res_list_temp_2[i][1]) + '\n')
out.close()

In [122]:
np.random.seed(57)
# add_res = 0
for t in range(1000000, 5000000):
    i, j = np.random.choice(1000, 2, replace=False)
    i_list, j_list = swap_greedy(i, j)
    now_happiness = sum([Children[c].happiness(i) for c in Gifts_list[i]]) + sum([Children[c].happiness(j) for c in Gifts_list[j]])
    new_happiness = sum([Children[c].happiness(i) for c in i_list]) + sum([Children[c].happiness(j) for c in j_list])
    
    if now_happiness < new_happiness:
        add_res += new_happiness - now_happiness
        Gifts_list[i] = i_list
        Gifts_list[j] = j_list
        
    if t % 10000 == 0:
        print("t = " + str(t) +" : increased " + str(add_res))

t = 1000000 : increased 205011011
t = 1010000 : increased 205235712
t = 1020000 : increased 205469130
t = 1030000 : increased 205684384
t = 1040000 : increased 205902471
t = 1050000 : increased 206114324
t = 1060000 : increased 206295470
t = 1070000 : increased 206486368
t = 1080000 : increased 206657024
t = 1090000 : increased 206841674
t = 1100000 : increased 207019791
t = 1110000 : increased 207193405
t = 1120000 : increased 207364218
t = 1130000 : increased 207516812
t = 1140000 : increased 207680742
t = 1150000 : increased 207832083
t = 1160000 : increased 207984914
t = 1170000 : increased 208132280
t = 1180000 : increased 208259657
t = 1190000 : increased 208409047
t = 1200000 : increased 208534526
t = 1210000 : increased 208665004
t = 1220000 : increased 208819497
t = 1230000 : increased 208956921
t = 1240000 : increased 209073218
t = 1250000 : increased 209210129
t = 1260000 : increased 209333467
t = 1270000 : increased 209463703
t = 1280000 : increased 209586385
t = 1290000 : 

In [123]:
np.random.seed(57)
# add_res = 0
for t in range(5000000, 10000000):
    i, j = np.random.choice(1000, 2, replace=False)
    i_list, j_list = swap_greedy(i, j)
    now_happiness = sum([Children[c].happiness(i) for c in Gifts_list[i]]) + sum([Children[c].happiness(j) for c in Gifts_list[j]])
    new_happiness = sum([Children[c].happiness(i) for c in i_list]) + sum([Children[c].happiness(j) for c in j_list])
    
    if now_happiness < new_happiness:
        add_res += new_happiness - now_happiness
        Gifts_list[i] = i_list
        Gifts_list[j] = j_list
        
    if t % 10000 == 0:
        print("t = " + str(t) +" : increased " + str(add_res))

t = 5000000 : increased 227303396
t = 5010000 : increased 227307816
t = 5020000 : increased 227311458
t = 5030000 : increased 227315021
t = 5040000 : increased 227318632
t = 5050000 : increased 227320158
t = 5060000 : increased 227323119
t = 5070000 : increased 227327894
t = 5080000 : increased 227331189
t = 5090000 : increased 227334149
t = 5100000 : increased 227336848
t = 5110000 : increased 227340522
t = 5120000 : increased 227344282
t = 5130000 : increased 227348348
t = 5140000 : increased 227353050
t = 5150000 : increased 227357048
t = 5160000 : increased 227359761
t = 5170000 : increased 227364107
t = 5180000 : increased 227366826
t = 5190000 : increased 227368477
t = 5200000 : increased 227371248
t = 5210000 : increased 227372711
t = 5220000 : increased 227375330
t = 5230000 : increased 227377390
t = 5240000 : increased 227380875
t = 5250000 : increased 227386173
t = 5260000 : increased 227389259
t = 5270000 : increased 227390836
t = 5280000 : increased 227392356
t = 5290000 : 

In [125]:
(1633769063 + add_res) / 2000000000

0.930791053

In [164]:
res_list_temp = [[] for i in range(1000000)]
for j in range(1000):
    for i in Gifts_list[j]:
        res_list_temp[i] = [i, j]

In [155]:
avg_normalized_happiness_(res_list_temp, child_pref, gift_pref)

normalized child happiness= 0.8620806 , normalized gift happiness 0.0687007805


0.93078138049999992

In [165]:
Gifts_list_2 = Gifts_list.copy()

In [166]:
# modify for twins
gain_move = 0
for i in range(2000):
    k1 = res_list_temp[2*i][1]
    k2 = res_list_temp[2*i+1][1]
    # which to go
    if k1 == k2:
        pass
    else:
        # 2*i move
        gain1 = Children[2*i].happiness(k2) - Children[2*i].happiness(k1)
        gain1_add = -10000000
        for l in Gifts_list_2[k2]:
            gain1_add_ = Children[l].happiness(k1) - Children[l].happiness(k2)
            if gain1_add_ > gain1_add and l > 2*i+1:
                v1 = l
                gain1_add = gain1_add_
        # 2*i+1 move
        gain2 = Children[2*i].happiness(k1) - Children[2*i].happiness(k2)
        gain2_add = -10000000
        for l in Gifts_list_2[k1]:
            gain2_add_ = Children[l].happiness(k2) - Children[l].happiness(k1)
            if gain2_add_ > gain2_add and l > 2*i+1:
                v2 = l
                gain2_add = gain2_add_
                
        if gain1 + gain1_add >= gain2 + gain2_add:
            res_list_temp[2*i][1] = k2
            res_list_temp[v1][1] = k1
            Gifts_list_2[k1].remove(2*i)
            Gifts_list_2[k2].append(2*i)
            Gifts_list_2[k2].remove(v1)
            Gifts_list_2[k1].append(v1)
        else:
            res_list_temp[2*i+1][1] = k1
            res_list_temp[v2][1] = k2
            Gifts_list_2[k2].remove(2*i+1)
            Gifts_list_2[k1].append(2*i+1)
            Gifts_list_2[k1].remove(v2)
            Gifts_list_2[k2].append(v2)
        gain_move += max(gain1 + gain1_add, gain2 + gain2_add)
print(gain_move)


0


In [163]:
avg_normalized_happiness(res_list_temp, child_pref, gift_pref)

normalized child happiness= 0.8618896 , normalized gift happiness 0.0685672215


0.93045682149999998

In [167]:
out = open('../submit/subm_hrd1214_10000000.csv', 'w')
out.write('ChildId,GiftId\n')
for i in range(len(res_list_temp)):
    out.write(str(i) + ',' + str(res_list_temp[i][1]) + '\n')
out.close()

以下はただの実験場

In [111]:
diff_child(0, 0, 1)

0.0

In [112]:
diff_child(1, 0, 1)

0.0

In [110]:
for i in range(2000):
    if res_list_temp[i*2][1] != res_list_temp[i*2+1][1]:
        print(i)

0
1
3
4
7
8
9
11
12
13
14
16
17
18
19
20
22
23
24
25
26
33
34
35
36
37
38
39
40
41
42
43
45
46
49
50
51
52
56
57
58
60
61
63
64
65
66
67
71
72
73
74
76
77
78
80
82
83
85
86
87
88
89
90
91
92
94
95
97
98
99
100
101
103
104
105
106
108
109
110
111
113
116
117
118
119
120
122
123
124
125
126
128
132
133
134
135
136
137
138
139
141
142
143
145
146
148
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
170
171
173
174
175
176
178
180
181
182
184
185
187
188
189
190
191
192
195
199
200
201
202
203
204
205
207
208
209
210
213
216
217
218
219
220
221
223
229
230
232
233
235
237
238
239
241
242
243
244
245
246
248
249
250
252
253
254
257
258
259
260
261
262
264
265
267
268
269
270
271
276
277
279
280
281
282
283
286
288
289
292
294
297
299
300
301
304
306
307
308
310
311
313
314
315
316
317
319
320
321
322
323
324
325
327
328
329
330
331
332
334
335
336
339
340
341
342
343
345
347
348
349
350
352
355
356
357
358
359
360
361
363
364
365
366
367
368
372
373
374
375
376
37

In [90]:
%%timeit
i = 0
j = 1
i_list, j_list = swap_greedy(i, j)
now_happiness = sum([Children[c].happiness(i) for c in Gifts_list[i]]) + sum([Children[c].happiness(j) for c in Gifts_list[j]])
new_happiness = sum([Children[c].happiness(i) for c in i_list]) + sum([Children[c].happiness(j) for c in j_list])

100 loops, best of 3: 5.12 ms per loop


In [101]:
%%timeit
i, j = np.random.choice(5, 2, replace=False)

The slowest run took 4.51 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 24.1 µs per loop


In [100]:
i

1

In [87]:
current, new

(3666329, 3666329)

In [78]:
all_users

[348,
 349,
 654,
 655,
 1254,
 1255,
 1594,
 1595,
 2786,
 2787,
 2878,
 2879,
 2990,
 2991,
 4754,
 5952,
 8080,
 8310,
 9153,
 9549,
 10041,
 10179,
 10363,
 11247,
 11579,
 12268,
 12823,
 13048,
 13240,
 14071,
 14807,
 15498,
 16271,
 17171,
 17939,
 18586,
 19049,
 19499,
 19825,
 20134,
 20612,
 21240,
 21433,
 22776,
 25791,
 26468,
 26562,
 26725,
 26986,
 29862,
 32194,
 32640,
 33947,
 34195,
 35217,
 35639,
 35778,
 35800,
 36705,
 37054,
 38288,
 38642,
 38927,
 39076,
 39224,
 39892,
 41104,
 42077,
 42297,
 43020,
 44770,
 45441,
 47840,
 48104,
 48569,
 50315,
 50500,
 50660,
 50730,
 51947,
 52456,
 52488,
 53863,
 54359,
 54874,
 54898,
 55348,
 55417,
 56049,
 56615,
 56948,
 57494,
 58213,
 59294,
 61260,
 61937,
 63718,
 65119,
 65617,
 65938,
 67042,
 67339,
 67428,
 70028,
 70047,
 71875,
 72520,
 72606,
 73872,
 74167,
 75820,
 75891,
 75961,
 76165,
 76325,
 76715,
 77068,
 77096,
 77227,
 77300,
 77373,
 82823,
 84060,
 85951,
 86114,
 88135,
 88487,
 89306,


In [75]:
prefer_order[:1000]

array([1693, 1005, 1503, 1219, 1500, 1004, 1220, 1221, 1222, 1493, 1492,
       1491, 1223, 1224, 1487, 1486, 1484, 1482, 1504, 1225, 1218, 1217,
       1211, 1528, 1527, 1525, 1524, 1212, 1213, 1521, 1214, 1215, 1516,
       1515, 1514, 1513, 1512, 1216, 1509, 1506, 1480, 1477, 1226, 1229,
       1442, 1440, 1439, 1437, 1436, 1435, 1434, 1432, 1430, 1429, 1428,
       1425, 1424, 1423, 1421, 1419, 1448, 1530, 1451, 1453, 1473, 1472,
       1227, 1469, 1468, 1467, 1465, 1464, 1463, 1462, 1461, 1460, 1458,
       1457, 1456, 1228, 1454, 1452, 1210, 1209, 1208, 1181, 1182, 1183,
       1624, 1184, 1185, 1618, 1180, 1617, 1186, 1187, 1188, 1189, 1608,
       1607, 1190, 1231, 1630, 1179, 1178, 1663, 1661, 1170, 1655, 1171,
       1652, 1651, 1649, 1648, 1172, 1173, 1174, 1642, 1175, 1176, 1177,
       1634, 1601, 1191, 1192, 1596, 1565, 1563, 1562, 1561, 1200, 1558,
       1557, 1201, 1550, 1202, 1203, 1204, 1541, 1205, 1206, 1207, 1536,
       1566, 1417, 1567, 1571, 1595, 1594, 1592, 11

In [72]:
swap_greedy(0, 1)

TypeError: only integer arrays with one element can be converted to an index

In [61]:
np.argsort([n if n%2 == 0 else 1 for n in range(20)])[-1]

18

In [60]:
np.arange(20,0,-1)[np.argsort([n if n%2 == 0 else 1 for n in range(20)])]

array([20,  3,  5,  7,  9, 13, 11, 15, 17, 19,  1, 18, 16, 14, 12, 10,  8,
        6,  4,  2])

In [55]:
sum([sum([Children[i].happiness(j) for i in Gifts_list[j]]) for j in range(1000)])

1633769063

In [67]:
lll1 = [1, 2, 3, 4]
lll2 = [5, 6, 7, 8]

In [68]:
lll2.append(lll1.pop())

In [69]:
lll1

[1, 2, 3]

In [70]:
lll2

[5, 6, 7, 8, 4]

In [45]:
sum([g.happiness() for g in Gifts_list])

1633769063

In [31]:
16337805 / 20000000

0.81689025

In [32]:
sum([g.gift_happiness() for g in Gifts_list])

-11437

In [35]:
-11437 / 2000000000

-5.7185e-06

In [46]:
1633780500 - 11437

1633769063